Finding a Protein Motif 
Given: At most 15 UniProt Protein Database access IDs.
Return: For each protein possessing the N-glycosylation motif, output its given access ID followed by a list of locations in the protein string where the motif can be found.

In [ ]:

# for this excercise i got this error
# urllib.error.URLError: <urlopen error [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:997)>
# and i was not able to fix it 
# that's why on Rosalind i didn't complete the problem
# but i think it work if i could fix this error


import urllib.request as url
import re
from urllib.request import urlopen
from Bio import SeqIO 

ID = [] 

proteins = 'P00740_FA9_HUMAN\nP07987_GUX2_TRIRE\nP01042_KNH_HUMAN\nP39873_RNBR_BOVIN\nP11171_41_HUMAN\nQ3BN23\nA6UUD2\nP20840_SAG1_YEAST\nQ3ATP6\nB5FPF2\nP01047_KNL2_BOVIN\nA6LJ74\nB5FNU0'

protein_list = proteins.split('\n')

for protein in protein_list:
    
    link = "http://www.uniprot.org/uniprot/"+ protein +".FASTA"
    data = urlopen(link)                                       
    fasta = data.read().decode('utf-8', 'ignore')             
    with open('seq_file.fasta', 'a') as text_file:            
        text_file.write(fasta)                                
                                                              
    handle = open('seq_file.fasta', 'r')                          
    motifs = re.compile(r'(?=(N[^P][ST][^P]))')                   
    count = 0                                                     
    for record in SeqIO.parse(handle, 'fasta'):                   
        sequence = record.seq                                     
        positions = []                                            
    for m in re.finditer(motifs, str(sequence)):              
                positions.append(m.start() + 1)                               
    if len(positions) > 0:                                    
        print(ID[count])                                      
        print(' '.join(map(str, positions)))                  
        count += 1          

Open Reading Frames
Either strand of a DNA double helix can serve as the coding strand for RNA transcription. Hence, a given DNA string implies six total reading frames, or ways in which the same region of DNA can be translated into amino acids: three reading frames result from reading the string itself, whereas three more result from reading its reverse complement.

An open reading frame (ORF) is one which starts from the start codon and ends by stop codon, without any other stop codons in between. Thus, a candidate protein string is derived by translating an open reading frame into amino acids until a stop codon is reached.

Given: A DNA string s of length at most 1 kbp in FASTA format.

Return: Every distinct candidate protein string that can be translated from ORFs of s. Strings can be returned in any order.

In [ ]:
# I don't know how many attempts i made a this problem but i managed to do it right in the end

 
def translate(rnaseq):
    codon_table = { 'UUU': 'F', 'CUU': 'L', 'AUU': 'I', 'GUU': 'V', \
                    'UUC': 'F', 'CUC': 'L', 'AUC': 'I', 'GUC': 'V', \
                    'UUA': 'L', 'CUA': 'L', 'AUA': 'I', 'GUA': 'V', \
                    'UUG': 'L', 'CUG': 'L', 'AUG': 'M', 'GUG': 'V', \
                    'UCU': 'S', 'CCU': 'P', 'ACU': 'T', 'GCU': 'A', \
                    'UCC': 'S', 'CCC': 'P', 'ACC': 'T', 'GCC': 'A', \
                    'UCA': 'S', 'CCA': 'P', 'ACA': 'T', 'GCA': 'A', \
                    'UCG': 'S', 'CCG': 'P', 'ACG': 'T', 'GCG': 'A', \
                    'UAU': 'Y', 'CAU': 'H', 'AAU': 'N', 'GAU': 'D', \
                    'UAC': 'Y', 'CAC': 'H', 'AAC': 'N', 'GAC': 'D', \
                    'UAA': 'Stop', 'CAA': 'Q', 'AAA': 'K', 'GAA': 'E', \
                    'UAG': 'Stop', 'CAG': 'Q', 'AAG': 'K', 'GAG': 'E', \
                    'UGU': 'C', 'CGU': 'R', 'AGU': 'S', 'GGU': 'G', \
                    'UGC': 'C', 'CGC': 'R', 'AGC': 'S', 'GGC': 'G', \
                    'UGA': 'Stop', 'CGA': 'R', 'AGA': 'R', 'GGA': 'G', \
                    'UGG': 'W', 'CGG': 'R', 'AGG': 'R', 'GGG': 'G'}

    length = len(rnaseq)
    proseq = []
    for i in range(0,length,3):
        triplet = rnaseq[i:i+3]
        if codon_table[str(triplet)] != 'Stop':
            proseq.append(codon_table[str(triplet)])
        else:
            break
    proseq = ''.join(proseq)
    return proseq

def rev_comp(x):
    tran = ''
    for i in range(len(x)):
        index = -(i+1)
        if x[index] == 'A':
            tran = tran + 'U'
        elif x[index] == 'T':
            tran = tran + 'A'
        elif x[index] == 'C':
            tran = tran + 'G'
        else:
            tran = tran +'C'
    revcomp = ''
    for i in range(len(tran)):
        index = -(i+1)
        if tran[index] == 'A':
            revcomp = revcomp + 'U'
        elif tran[index] == 'U':
            revcomp = revcomp + 'A'
        elif tran[index] == 'C':
            revcomp = revcomp + 'G'
        else:
            revcomp = revcomp +'C'
    return tran, revcomp
 
def find_all(s,substring):
    index_list = []
    index = s.find(substring)
    while index != -1:
        index_list.append(index)
        index = s.find(substring, index+1)
    if len(index_list) > 0:
        return index_list
    else:
        return -1
 
def orf(mrna):
    start_codon = 'AUG'
    stop_codon = ['UAA', 'UAG', 'UGA']
    i, j = 0,0
    out = []
    while i <= len(mrna)-2:
        if mrna[i:i+3] == start_codon:
            j=i
            sequence=''
            while i<= len(mrna) -2:
                if mrna[i:i+3] in stop_codon:
                    out.append(sequence)
                    break
                sequence = sequence + mrna[i:i+3]
                i = i+3
        i = j+1
        j = j+1
    return out  

dna = ''
mrna1, mrna2 = rev_comp(dna)
orf_list1=orf(mrna1)
orf_list2=orf(mrna2)
orf_list = orf_list1
for i in orf_list2:
    if i not in orf_list:
        orf_list.append(i)
for i in orf_list:
    print(translate(i))
 

Overlapping Graphs
A graph whose nodes have all been labeled can be represented by an adjacency list, in which each row of the list contains the two node labels corresponding to a unique edge.

A directed graph (or digraph) is a graph containing directed edges, each of which has an orientation. That is, a directed edge is represented by an arrow instead of a line segment; the starting and ending nodes of an edge form its tail and head, respectively. The directed edge with tail v and head w is represented by (v,w) (but not by (w,v)). A directed loop is a directed edge of the form (v,v).

For a collection of strings and a positive integer k, the overlap graph for the strings is a directed graph Ok in which each string is represented by a node, and string s is connected to string t with a directed edge when there is a length k suffix of s that matches a length k prefix of t, as long as s≠t; we demand s≠t to prevent directed loops in the overlap graph (although directed cycles may be present).

Given: A collection of DNA strings in FASTA format having total length at most 10 kbp.

Return: The adjacency list corresponding to O3. You may return edges in any order.

In [ ]:
from Bio import SeqIO

file = 'rosalind_grph.txt'

with open(file) as f:
    sequences = list(SeqIO.parse(f, 'fasta'))

for fasta1 in sequences:
    for fasta2 in sequences:
        name1, sequence1 = fasta1.id, str(fasta1.seq)
        name2, sequence2 = fasta2.id, str(fasta2.seq)
        if sequence1 == sequence2:
            continue
        suffix = sequence1[-3:]
        prefix = sequence2[:3]
        if prefix == suffix:
            print(name1, name2)

RNA Splicing
After identifying the exons and introns of an RNA string, we only need to delete the introns and concatenate the exons to form a new string ready for translation.

Given: A DNA string s (of length at most 1 kbp) and a collection of substrings of s acting as introns. All strings are given in FASTA format.

Return: A protein string resulting from transcribing and translating the exons of s. (Note: Only one solution will exist for the dataset provided.)



In [ ]:
rna2aa_dictionary =  {'UUU' : 'F', "UUC": 'F', 'UUA' : 'L', 'UUG' : 'L', 'UCU' : 'S' , 'UCA' :'S', 'UCC' : 'S', 'UCG' : 'S', 'UAU' : 'Y', 'UAC': 'Y', 'UAA': 'STOP' , 'UAG': 'STOP', 'UGU' : 'C', 'UGC': 'C', 'UGA' : 'STOP', 'UGG': 'W', 'CUU' : 'L', 'CUC' : 'L', 'CUA': 'L', 'CUG': 'L', 'CCU': 'P', 'CCC' : 'P', 'CCA' : 'P', 'CCG' : 'P', 'CAU' : 'H', 'CAC':'H', 'CAA':'Q','CAG':'Q','CGU': 'R','CGC':'R','CGA':'R','CGG':'R','AUU':'I','AUC':'I','AUA':'I','AUG':'M','ACU':'T','ACC':'T','ACA':'T','ACG':'T','AAU':'N','AAC':'N','AAA':'K','AAG':'K','AGU':'S','AGC':'S','AGA':'R','AGG':'R','GUU':'V','GUC':'V','GUA':'V','GUG':'V','GCU':'A','GCC':'A','GCA':'A','GCG':'A','GAU':'D','GAC':'D','GAA':'E','GAG':'E','GGU':'G','GGC':'G','GGA':'G','GGG':'G'}

file = open("rosalind_splc.txt", "r")

def extract_sequence(file):
    sequences = []
    result = ""       
    
    for seq in file:
        if ">" in seq:
            sequences.append(result)
            result = ""
        else:
            if "\n" in seq:
                result += seq[:len(seq) - 1]
            else:
                result += seq

    sequences.append(result)
    sequences.remove('')
    return sequences

sequences = extract_sequence(file)

dna_string = sequences[0]
introns = sequences[1:]
introns.sort(reverse = True)

for intron in introns:
    dna_string = dna_string.replace(intron,"")
dna_string = dna_string.replace("T","U")

for i in range(0, len(dna_string) - 3, 3):
    print(rna2aa_dictionary[dna_string[i: i +3]],end='')      


Finding a spliced Motif
A subsequence of a string is a collection of symbols contained in order (though not necessarily contiguously) in the string (e.g., ACG is a subsequence of TATGCTAAGATC). The indices of a subsequence are the positions in the string at which the symbols of the subsequence appear; thus, the indices of ACG in TATGCTAAGATC can be represented by (2, 5, 9).

As a substring can have multiple locations, a subsequence can have multiple collections of indices, and the same index can be reused in more than one appearance of the subsequence; for example, ACG is a subsequence of AACCGGTT in 8 different ways.

Given: Two DNA strings s and t (each of length at most 1 kbp) in FASTA format.

Return: One collection of indices of s in which the symbols of t appear as a subsequence of s. If multiple solutions exist, you may return any one.

In [ ]:

fasta_formatted_input = ">Rosalind_9160\nGATCGATAGGCAATTTGGCTCGGGAGAATTCTTGGTGTGCTCACTCCGATTCTCTTCCGATTAAGAGTTTGGTTCCTTGGGCAATGCAGCTTGCTCATGGGGGAAGATTAAGGGGAACTTTTGGCTATGAGACATAGACAGACCTTAGTGTGGCGTCTAGTCGGAATCTAGAGCCACACTAGTGTACATAATCGGGTGAGACATTACCGAAGACGGATCGCTGTTCGGGTAATGATGAATCCTGAAGATTGTCGTCTATGCTATGCTTCCGTTGCTCGGATCCGGGACTGACAAAAAAGCAGGGGAGGTGTTTAAAGGCTAAGCCTCCAAAAGGTGCAGAACATGACGAGCCTTCTATCGGGTAAGTGTTAAGGCGTCGTCCAGAATCTCGCATCTCGCTCTACGACTCTCGTTTGAAAACCAGTTGACGGACGCTCCTAGCCGCTACCGGAGGACATAATCTGGTTCAGGCGCCACCTACAGGTTCTGACACGTGGGACTTTAACTCGCCCCTAAAATCAGTAGACGAAACCGGTAAGGGTTCATTTCGGCCAAGGCGATGCACATCATCTACGGTTCCGAGGCTGGTGCTCGTCACGGCGATCCTAGATACCGCGACACCAGTGCAACCTGTGTGGTAGTGAAGAGGGCTGAGTCGACGCGCGGCGATCGCGTCGAAAACTACTTCCAGTACAACCAGGAAACCCTGGTGAAATTGTTTTGATAAACCCTTATATTTTCATGGTTAGGGCAAGAATGAACCCTAAGCCTTACACCCATCTGTGTCAACGGCGTTAGTGGCACTTAGTACCTGCCGTAACTGTGATATAAAGGTTTGTGAAGGAAAGGCCCACACTCTCTCGTTCTGGAGCGTTGACCAGAATGGCCAATCGGCTGATGCGCGCGCTACATGACATACGCACAAGTCTTTGCGATCGTATG\n>Rosalind_1858\nTTGCGGATGCTCCGCTAGGGTACCTCTCCCTTCCGAAGGTGCGGCTCACTCTAGCTCGTTT"
splitted_input = fasta_formatted_input.split("\n")

dna_sequence = splitted_input[1]
motif = splitted_input[3]

indices = ""
dna_index = 0
motif_index = 0

for base in dna_sequence:
    if base == motif[motif_index]:
        motif_index += 1
        indices += str(dna_index + 1) + " "
        if motif_index == len(motif):
            break
        
    dna_index += 1
print(indices)

Transitions and Transversions
For DNA strings s1 and s2 having the same length, their transition/transversion ratio R(s1,s2) is the ratio of the total number of transitions to the total number of transversions, where symbol substitutions are inferred from mismatched corresponding symbols as when calculating Hamming distance (see “Counting Point Mutations”).

Given: Two DNA strings s1 and s2 of equal length (at most 1 kbp).

Return: The transition/transversion ratio R(s1,s2).

In [3]:
fasta = ">Rosalind_8091\nTCCACGCCGCTTACGACTCATTGGTGATTTTGACTAGACGAGTAGTGAAAAAGCTATTCTCCGTCAACAGATGAGATGTTAGATAGATTGTTGTTCCTTTTCACTGTCAATTGGTATTGACTCCCCTTCCAAACCTTTTAGCATACATGACTGTCCTAGCTCCAGTACCGGTAGTGTAGAGTCAGACCCTCAAACATAAGTAAGAGACGATTTATTGACCGTAGCCGAGTCCAGCATAGTTTTTATCATTACGATTTCGATAAGATTATGCAAGGCCTCGCCTCGGGTAGAACGTATAGATGCTCTAACCGAGTTCCATGTCGGGACACACGCCGATCACACTAAACAGTATCATTTTATCTTAAAGTGAGTCCTGGCTGCTACGGGATTATGCGCGCGGGTCACACGATCGAGAGACAGTACAAGCACACGAAAGAGACACATGCGCAAGCCCACCGCGGAACGACAAATATTTGTAATACGCCATAGCGGCTTATCCAACCGGAATGCGCGAAGCATGCCACGCTGCTCCAAGGCTCGAGTTCCGACACTCAACTCAACGGACGGGGGGGCTGCTAGGCTCTAACCCGGCTGACGCAAGGATGGCACCCGGAGAACCACGCATTTGTTCCTGGTCTTAACTAACTTAGGCATTGCTTTCAGTTACGGCCAGCATCTTTTCAGAGTCTCCCTGGGTGTAGCATACCCAAAACAACTGGTGTTCGCCTCAAGGTAGCGTAGGAATAACTTCAGAGAAATTACTACCCAGCGACTCGTACCGGGAGTGTTAGGTTCGAAATTGCCCGCACCGCTGTGGGCTAGGGGCACATATATGGCTTTTCTTTGCATACCGGAGTTCACGGTTAACGGCTACCCAAGCTCTTAATGCTTGAATCGTAACGCCAACCCTGCCCGAAGCTGGGATTGTTGTATGGTAATTGAA\n>Rosalind_3665\nTCCACACCGCTAACAACTCACTAGTCACTCCGACTCGACAAGTGGTGTAGTAGATAACCTCCGTCAACAGAGGAGATGTTGGATAAATCGTACTTCCTTATTACTGTCAGGTGGTATTGATACCCCTTCGGAACCTTATAGTAGACGTGGGCATTCTAGTTCCAGCGCTAACACTATCGAGCTGCACTCTCAATCATGAGTAAGAGACAGCTTACTAACCGTGGCTGAGTCCGGGTTATTTTGAATGTTTACGATTTCGATGAGATTATGTGAAGCTTGACTTCGAAGAAAACGCCTAGATGCTCTAACCGGGTTCCATATCCCGAGTCATGCTGATCACATTAACTACTACCAATCGATCTTGGAACAAGTCCGGGCCCCTGCACAATGATGCACACGAGTCATACGATCGGGAGACAGTACAAACACGTGAAAAGGCAACGTATGCAAGTCCACCGCAAAGTAACATAAATTGGTAATGCGCCATAGCGGCTTACTTAACCAGAATCTGCGAGGCACACCGGGTTTTTCCAAGGGCCGAGCCCCGAAAGTCGATTCGAGGGACGGGGGGGCTGCTAGGGTCAAATCCGGTTGACGTAGAGGGGGTGACGGACGAACCACGCAGCCGTGCCAGACCTTGACTAACTTAGGTATTATTGTAAGCCACGGCCATTATCTTTACAGGGTTTCTCTGGGTGTGATGTACCCAAAATTACCGGCGTACGCCTCAAAACAGTTCAGGTATAGACTCACAGAAATTGCTCCCAAGCGACATGCACCGGGAGCGCCAGGTGCGGAACTACACGTGCCGCAATCAGCTCGGAGTACATGTATCATTTACTTTCGCAAGCCGGAGTTCACGGTTAACAGCTATTCAGAGTCTCAGTGATTGCATCCTGACGCTATCCTTGCTTGAAACTGGGAAGGTCGCACCGCAGCTAAC"
splitted = fasta.split("\n")
first_string = splitted[1]
second_string = splitted[3]

def tran(base1, base2):
    return (base1 + base2) in ["AG","GA","CT","TC"]

transition = 0
transversion = 0

for i in range(len(first_string)):
    first_base, second_base = first_string[i], second_string[i]
    
    if first_base != second_base:
        if tran(first_base, second_base):
            transition += 1
        else:
            transversion += 1

ratio = (transition / transversion)

print(ratio)

2.0106382978723403


Completing a Tree
An undirected graph is connected if there is a path connecting any two nodes. A tree is a connected (undirected) graph containing no cycles; this definition forces the tree to have a branching structure organized around a central core of nodes, just like its living counterpart. See Figure 2.

We have already grown familiar with trees in “Mendel's First Law”, where we introduced the probability tree diagram to visualize the outcomes of a random variable.

In the creation of a phylogeny, taxa are encoded by the tree's leaves, or nodes having degree 1. A node of a tree having degree larger than 1 is called an internal node.

Given: A positive integer n (n≤1000) and an adjacency list corresponding to a graph on n nodes that contains no cycles.

Return: The minimum number of edges that can be added to the graph to produce a tree.

In [1]:
# I had to try any times because i couldn't figure out why it wasn't working 
# but then I noticed that the problem was the -1 missing

input = "rosalind_tree.txt"

with(open(input, 'r')) as f:
    n = int(f.readline())
    nodes = [line.split() for line in f]

edges = (n - len(nodes) - 1)
print(edges)

33


Genome Assembly as Shortest Superstring
For a collection of strings, a larger string containing every one of the smaller strings as a substring is called a superstring.

By the assumption of parsimony, a shortest possible superstring over a collection of reads serves as a candidate chromosome.

Given: At most 50 DNA strings of approximately equal length, not exceeding 1 kbp, in FASTA format (which represent reads deriving from the same strand of a single linear chromosome).

The dataset is guaranteed to satisfy the following condition: there exists a unique way to reconstruct the entire chromosome from these reads by gluing together pairs of reads that overlap by more than half their length.

Return: A shortest superstring containing all the given strings (thus corresponding to a reconstructed chromosome).

In [ ]:
# I found this code pretty difficult so I helped myself with the internet 

from Bio import SeqIO
import io

def get_superstring(reads_list, superstring=''):
    if len(reads_list) == 0:
        return superstring

    elif len(superstring) == 0:
        superstring = reads_list.pop(0)
        return get_superstring(reads_list, superstring)

    else:
        for current_read_index in range(len(reads_list)):
            current_read = reads_list[current_read_index]
            current_read_length = len(current_read)

            for trial in range(current_read_length // 2):
                overlap_length = current_read_length - trial

                if superstring.startswith(current_read[trial:]):
                    reads_list.pop(current_read_index)
                    return get_superstring(reads_list, current_read[:trial] + superstring)

                if superstring.endswith(current_read[:overlap_length]):
                    reads_list.pop(current_read_index)
                    return get_superstring(reads_list, superstring + current_read[overlap_length:])


if __name__ == "__main__":

    small_dataset = """
"""
    handle = io.StringIO(small_dataset)

    reads = []
    for record in SeqIO.parse(handle, 'fasta'):
        reads.append(str(record.seq))
    handle.close()
    ov = get_superstring(reads)
    print(ov)

Partial Permutations
A partial permutation is an ordering of only k objects taken from a collection containing n objects (i.e., k≤n). For example, one partial permutation of three of the first eight positive integers is given by (5,7,2).

The statistic P(n,k) counts the total number of partial permutations of k objects that can be formed from a collection of n objects. Note that P(n,n) is just the number of permutations of n objects, which we found to be equal to n!=n(n−1)(n−2)⋯(3)(2) in “Enumerating Gene Orders”.

Given: Positive integers n and k such that 100≥n>0 and 10≥k>0.

Return: The total number of partial permutations P(n,k), modulo 1,000,000.

In [ ]:
n = 93
k = 10

partial_perm = 1
for i in range(k):
    partial_perm *= (n - i)
print(partial_perm % 1000000)

Perfect Matchings and RNA Secondary Structures

Given: An RNA string s of length at most 80 bp having the same number of occurrences of 'A' as 'U' and the same number of occurrences of 'C' as 'G'.

Return: The total possible number of perfect matchings of basepair edges in the bonding graph of s.

In [ ]:
from Bio import SeqIO                      
from math import factorial 

sequence = ''                              
fasta = open('rosalind_pmch.txt', 'r')     
for x in SeqIO.parse(fasta, 'fasta'):
    sequence = str(x.seq)             
fasta.close()                             

AU = 0                                     
GC = 0                                     
for k in sequence:                        
    if k == 'A':                          
        AU += 1                            
    elif k == 'G':                        
        GC += 1                            

match = factorial(AU) * factorial(GC)  
print(match)               